# HVNM Tutorial 01: Flow Curve NLSQ→NUTS

## Introduction

At steady state under constant shear rate $\dot{\gamma}$, the HVNM E-network and I-network stresses vanish ($\sigma_E = \sigma_I = 0$) because natural states track the deformation via TST kinetics. Only the **D-network provides viscous dissipation**: $\sigma_{ss} = \eta_D \dot{\gamma}$ with $\eta_D = G_D / k_{d,D}$. The flow curve therefore constrains exactly **2 parameters**: $G_D$ (modulus) and $k_{d,D}$ (rate). This tutorial fits ethyl cellulose 7% shear-thinning data using NLSQ for point estimates, then NUTS for full Bayesian posteriors with ArviZ diagnostics (R-hat, ESS, posterior predictive checks).

> **Handbook:** See [HVNM Protocol Derivations](../../docs/source/models/hvnm/hvnm_protocols.rst) for steady-state analytical solution and [Knowledge Extraction](../../docs/source/models/hvnm/hvnm_knowledge.rst) for $\phi$-independence validation.

## Learning Objectives

- Fit steady shear flow curve with HVNM (2-parameter inference)
- Use log-space residuals for decades-spanning data
- Warm-start NUTS with NLSQ estimates
- Interpret posteriors: $\tau_D = 1/k_{d,D}$, $\eta_D = G_D / k_{d,D}$

## Prerequisites
- **Notebook 08** — Data intake and QC workflows

## Estimated Runtime
- NLSQ: ~5 s | NUTS: ~30 s (FAST_MODE) / ~5 min (production)

## 1. Setup

In [ ]:
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax openpyxl
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models import HVNMLocal

jax, jnp = safe_import_jax()
verify_float64()

sys.path.insert(0, "../..")
from examples.utils.hvnm_tutorial_utils import (
    configure_hvnm_for_fit,
    get_bayesian_config,
    get_fast_mode,
    get_nlsq_values,
    get_output_dir,
    load_ec_flow_curve,
    plot_fit_comparison,
    plot_ppc,
    plot_trace_and_forest,
    print_convergence,
    print_parameter_table,
    save_figure,
    save_results,
    setup_style,
)

setup_style()
print(f"JAX {jax.__version__}, devices: {jax.devices()}")
print(f"FAST_MODE: {get_fast_mode()}")
import os
import sys

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

## 2. Load and Inspect Data

In [ ]:
data = load_ec_flow_curve("07-00")
print(data.summary())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.loglog(data.x, data.y, 'o', ms=5, color='steelblue')
ax1.set_xlabel(data.x_label)
ax1.set_ylabel(data.y_label)
ax1.set_title('Stress vs Shear Rate')
ax1.grid(True, alpha=0.3, which='both')

ax2.loglog(data.x, data.y2, 'o', ms=5, color='coral')
ax2.set_xlabel(data.x_label)
ax2.set_ylabel(data.y2_label)
ax2.set_title('Viscosity vs Shear Rate')
ax2.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.show()

## 3. HVNM Model Configuration

For the flow curve, only `G_D` and `k_d_D` are identifiable. All other HVNM
parameters are fixed at defaults (they don't affect steady-state stress).

In [ ]:
model = HVNMLocal(include_dissociative=True)
fit_params = configure_hvnm_for_fit(
    model,
    protocol="flow_curve",
    overrides={
        "G_D": 10.0,     # Initial guess (will be fitted)
        "k_d_D": 100.0,  # Initial guess
    },
)

print(f"Fittable parameters: {fit_params}")
print(f"\nInitial values:")
for p in fit_params:
    val = model.parameters.get_value(p)
    lo, hi = model.parameters[p].bounds
    print(f"  {p}: {val:.4g}  bounds=[{lo:.4g}, {hi:.4g}]")

## 4. NLSQ Fitting

We use log-space residuals for the flow curve (standard practice for data
spanning multiple decades).

In [ ]:
t0 = time.time()
model.fit(
    data.x_masked,
    data.y_masked,
    test_mode="flow_curve",
    use_log_residuals=True,
    max_iter=2000,
)
nlsq_time = time.time() - t0

nlsq_vals = get_nlsq_values(model, fit_params)
print(f"\nNLSQ completed in {nlsq_time:.1f} s")
print(f"Fitted parameters:")
for p, v in nlsq_vals.items():
    print(f"  {p} = {v:.6g}")

# Physical interpretation
tau_D = 1.0 / nlsq_vals['k_d_D']
eta_D = nlsq_vals['G_D'] / nlsq_vals['k_d_D']
print(f"\nDerived quantities:")
print(f"  tau_D = 1/k_d_D = {tau_D:.4g} s")
print(f"  eta_D = G_D/k_d_D = {eta_D:.4g} Pa.s")

In [ ]:
fig = plot_fit_comparison(data, model, title="HVNM Flow Curve: NLSQ Fit")
save_figure(fig, "hvnm_01_flow_nlsq_fit.png")
plt.show()

## 5. Bayesian Inference (NUTS)

We use the NLSQ result as a warm-start for NumPyro NUTS sampling.
The priors are consistent with the parameter bounds in the ParameterSet.

In [ ]:
bayes_cfg = get_bayesian_config()
print(f"Bayesian config: {bayes_cfg}")

t0 = time.time()
result = model.fit_bayesian(
    data.x_masked,
    data.y_masked,
    test_mode="flow_curve",
    **bayes_cfg,
)
nuts_time = time.time() - t0
print(f"\nNUTS completed in {nuts_time:.1f} s")

## 6. Convergence Diagnostics

In [ ]:
converged = print_convergence(result, fit_params)
print()
print_parameter_table(fit_params, nlsq_vals, result.posterior_samples)

In [ ]:
display_arviz_diagnostics(result, fit_params, fast_mode=get_fast_mode())

## 7. Posterior Predictive Check

In [ ]:
fig = plot_ppc(
    data, model, result.posterior_samples, fit_params,
    title="HVNM Flow Curve: Posterior Predictive Check",
)
save_figure(fig, "hvnm_01_flow_ppc.png")
plt.show()

## 8. Save Results

In [ ]:
out_dir = get_output_dir("flow_curve")
save_results(
    out_dir, model, result,
    param_names=fit_params,
    extra_meta={"dataset": "EC_7pct", "protocol": "flow_curve"},
)

## Key Takeaways

1. **Flow curve constrains 2 params**: $G_D$ and $k_{d,D}$ only (E/I networks vanish at SS)
2. **Log-space residuals**: Standard for decades-spanning rheological data
3. **NLSQ warm-start**: Dramatically improves NUTS efficiency (fewer divergences)
4. **Derived quantities**: $\tau_D = 1/k_{d,D}$ (relaxation time), $\eta_D = G_D / k_{d,D}$ (zero-shear viscosity)
5. **Cross-protocol**: Use with SAOS (NB13) or global fitting (NB15) to constrain E/I networks

## Further Reading

**Handbook Documentation:**
- [HVNM Protocol Derivations](../../docs/source/models/hvnm/hvnm_protocols.rst) — Steady-state analytical solution
- [HVNM Knowledge Extraction](../../docs/source/models/hvnm/hvnm_knowledge.rst) — Why $\phi$-independent, cross-protocol workflows

**Key References:**
1. Karim, M.R., Vernerey, F. & Sain, T. (2025). *Macromolecules*, 58(10), 4899-4912.

## Next Notebooks

- **Notebook 10**: Creep compliance NLSQ/NUTS
- **Notebook 13**: SAOS NLSQ/NUTS — Full network moduli
- **Notebook 15**: Global multi-protocol — Joint inference